In [1]:
import re
import pickle 
from itertools import chain
from datetime import datetime
from collections import defaultdict

from typing import List, Dict, Optional, Iterable, Tuple

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing

from ngram_model import Tokenizer, GenerationConfig, StatLM, construct_model

In [21]:
generation_config = GenerationConfig(temperature = 0.4, max_tokens = 32,
                                     sample_top_p = 0.9, decoding_strategy = 'top-p',
                                     remove_special_tokens=True)

In [3]:
dataset_jokes = load_dataset('IgorVolochay/russian_jokes')
dataset_dialogues = load_dataset('Den4ikAI/russian_dialogues')

In [10]:
train_texts_jokes = dataset_jokes['train']['text']
train_texts_dialogues_q = dataset_dialogues['train']['question']
train_texts_dialogues_a = dataset_dialogues['train']['answer']
train_texts_dialogues = []

In [13]:
for i in range(len(train_texts_dialogues_a)):
    if (train_texts_dialogues_a[i] != None and train_texts_dialogues_q[i] != None):
        train_texts_dialogues.append(train_texts_dialogues_q[i] + ' ' + train_texts_dialogues_a[i])

In [14]:
train_texts_dialogues

['как дела? там хорошо',
 'вы кефир пачему не кушаете, не любите? я ряженку лучше люблю.',
 'если в расходную накладную забить дури и выкурить, то получится приходный документ? особенно когда придет комиссия проверять документацию',
 'покажись в шапке ды щаз приветик',
 'давай не будем об этом давай поговорим о чем-нибудь другом',
 'препарат для лечения сильно понижает давление. что порекомендуете? чтоб не сильно? или что? препарат принимай и кофе пей',
 'мужчина, если ты занюхиваешь волосами соседки, то какой аромат предпочитаешь? предпочитаю соседкиными пирогами закусывать. -',
 'можете ли вы с ходу отличить японских фигуристов от китайских без представления? японцы в целом куда симпатичнее китайцев. но конечно же бывают исключения во всем',
 'как стать наемником? свяжись с нанимателями, заключи договор и будь наемником.',
 'слышала, что есть крем от шрамов. кто знает, скажите пожалуйста название. и поможет ли он от шрамов которым лет. дцать? контроктубекс и кажется медерма? фигня по

In [15]:
tokenizer = Tokenizer().build_vocab(train_texts_dialogues)

In [25]:
# класс, который позволяем строить и использовать языковую модель на основе n-грамм
stat_lm = StatLM(tokenizer, context_size=10, alpha=0.1) # , sample_top_p = None

# "обучаем" модель - считаем статистики
stat_lm.train(train_texts_dialogues)

train lines:   0%|          | 0/2476535 [00:00<?, ?it/s]

In [28]:
tokenizer.save('models/stat_lm/tokenizer.pkl')
stat_lm.save_stat('models/stat_lm/stat_lm.pkl')

True

In [26]:
stat_lm.generate("Видел недавно на дваче тред где чувак ", generation_config)

'видел недавно на дваче тред где чувак загаданныйм рубилово циклоалканы извинилась спиридона козленочком уженереально отобрано нарциссическим облазит обратных дадачто егорушка нулевой отделаем отвращаются очищены опредить трулала лакомить киз актара кросовом wtfнастя переливающимися'

In [27]:
model, kwargs = construct_model()

model.generate("Расскажи анекдот про поручика Ржевского", **kwargs)

'расскажи анекдот про ржевского сочиненья нефизического метобализм инопланетном йок неделимость радоти неясно спускало безворсовую таблерон коронографии поздравляюкакую чатуалексей щасье банов кубанскую напущенной пожалйуста деревье postpaid репетицией интересоввот пентагона ключарев цветочнъй толстокожей'